# Criando um banco de dados e inserindo dados

## Criando um banco de dados

Para o banco de dados, utilizamos o MySQL, criando 3 tabelas, genes, variantes e evidencias. A tabela genes contém os genes que foram analisados, a tabela variantes contém as variantes encontradas e a tabela evidencias contém as evidências encontradas para cada variante.

<img src="https://raw.githubusercontent.com/perolatai/projeto_CDIA_CIVIC/main/database/civic_erd.png">

In [ ]:
CREATE TABLE genes
(
    entrez_id INTEGER PRIMARY KEY,
    gene VARCHAR(255),
    description TEXT
);

CREATE TABLE variantes
(
    variant_id INTEGER PRIMARY KEY,
    variant VARCHAR(255),
    summary TEXT,
    start VARCHAR(255),
    stop VARCHAR(255),
    reference_bases VARCHAR(255),
    variant_bases VARCHAR(255),
    representative_transcript VARCHAR(255),
    ensembl_version FLOAT,
    reference_build VARCHAR(255),
    variant_types VARCHAR(255),
    hgvs_expressions VARCHAR(255),
    civic_variant_evidence_score FLOAT,
    allele_registry_id VARCHAR(255),
    clinvar_ids VARCHAR(255),
    variant_aliases VARCHAR(255)
);

CREATE TABLE evidencias
(
    evidence_id INTEGER PRIMARY KEY,
    entrez_id INTEGER,
    disease VARCHAR(255),
    doid FLOAT,
    phenotypes VARCHAR(255),
    drugs VARCHAR(255),
    drug_interaction_type VARCHAR(255),
    evidence_type VARCHAR(255),
    evidence_direction VARCHAR(255),
    evidence_level VARCHAR(255),
    clinical_significance VARCHAR(255),
    evidence_statement TEXT,
    citation_id INTEGER,
    source_type VARCHAR(255),
    citation VARCHAR(255),
    rating FLOAT,
    evidence_status VARCHAR(255),
    variant_id INTEGER,
    chromosome VARCHAR(255),
    variant_origin VARCHAR(255),
    is_flagged BOOLEAN,
    FOREIGN KEY (entrez_id) REFERENCES genes(entrez_id),
    FOREIGN KEY (variant_id) REFERENCES variantes(variant_id)
);

A tabela evidencias possui uma chave estrangeira para a tabela variantes, que é a variante_id e uma chave estrangeira para a tabela genes, que é a gene_id, possibilitando a ligação entre as tabelas.

## Inserindo dados

Ao extrairmosos dados da API, utilizamos os arquivos para inserir os dados em nosso banco de dados, atraves de um script em python.

In [ ]:
def insert_dataframe_into_mysql(df, table_name, connection):
    cursor = connection.cursor()
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    for _, row in df.iterrows():
        row = pd.concat([pd.Series([0]), row])
        row = row.iloc[1:]
        values = tuple(row.fillna('None'))
        cursor.execute(sql, values)
    connection.commit()


insert_dataframe_into_mysql(df_Gene, 'genes', conn)
insert_dataframe_into_mysql(df_Variant, 'variantes', conn)
insert_dataframe_into_mysql(df_Evidences, 'evidencias', conn)

## Disponibilizando os dados

Para que o grupo tenha acesso a todo momento aos dados, disponibilizamos um banco de dados na nuvem, para que possam acessar e fazer consultas, através do site [https://www.elephantsql.com/](https://www.elephantsql.com/), que utilizam o PostgreSQL como banco de dados.

Para acessarmos os dados, utilizamos o seguitne comando:

In [ ]:
conn = psycopg2.connect(
    database=database,
    host=host,
    user=user,
    password=password,
    port=port)